In [8]:
import pandas as pd
import numpy as np
import pandas_profiling
import plotly 
import cufflinks as cf
import plotly.offline
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go

In [12]:
data = pd.read_csv("../data/csv_data_keylogger.csv")
data.head()

,key_code,timing_ms,absolut_ms,coord_X,coord_Y
0,RBUTTON,11640,11636,852,329
1,LBUTTON,2563,15742,614,531
2,t,2375,19466,NaN,NaN
3,e,156,19661,NaN,NaN
4,s,859,20546,NaN,NaN


<div class="alert alert-block alert-info">
<b>Чистим данные</b> от больших перерывов между набором 
убираем системные прерывания
Удаляем шелчки мыши</div>

In [16]:
data['key_code'] = data['key_code'].dropna()
data['timing_ms'] = data['timing_ms'].dropna()
data['absolut_ms'] = data['absolut_ms'].dropna()
data = data.loc[data['key_code'] != "key_code"]
data = data.loc[data['key_code'] != " "]
data = data.loc[data['timing_ms'] != " "]
data = data.loc[data['absolut_ms'] != " "]
data = data.loc[data['key_code'] != "LBUTTON"]
data = data.loc[data['key_code'] != "RBUTTON"]
data = data.loc[data['key_code'] != "MBUTTON"]

for i in range(len(data)):
    data.iloc[i,1] = int(data.iloc[i,1])
    
for i in range(len(data)):
    data.iloc[i,2] = int(data.iloc[i,2])
data['absolut_ms'] = data['absolut_ms'] - min(data['absolut_ms'])

data = data[np.array(data['timing_ms']) < 1000.0]
data.head()

,key_code,timing_ms,absolut_ms,coord_X,coord_Y
3,e,156,0,NaN,NaN
4,s,859,885,NaN,NaN
5,t,328,1249,NaN,NaN
6,NaN,344,1622,NaN,NaN
7,D,844,2489,NaN,NaN


<div class="alert alert-block alert-warning">
<b>Общий отчёт</b> (интерактив)
</div>

In [17]:
pandas_profiling.ProfileReport(data)

<div class="alert alert-block alert-warning">
<b>Графики</b> (интерактив)
</div>

In [21]:
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
data.iplot()

<div class="alert alert-block alert-success">
<b>То же самое, но крупнее и с одной колонкой 'key_code'</b> (интерактив)
</div>

In [19]:
init_notebook_mode(connected = True)

def plotly_df(df, title = ''):
    data = []
    trace = go.Scatter(x = df['absolut_ms'],y = df['timing_ms'],mode = 'lines',name = 'key_code')
    data.append(trace)
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

plotly_df(data, title = "Press delay relative to time")

In [22]:
d_key_count = data.groupby(['key_code']).size().reset_index(name='count')
d_key_count = d_key_count.sort_values(['count'], ascending=False)
d_key_count.head()

,key_code,count
37,t,20
22,e,17
23,f,16
31,n,14
41,y,14
